# 1. 计算图

通过数值微分计算了神经网络的权重参数的梯度（严格来说，是损失函数关于权重参数的梯度）。数值微分虽然简单，也容易实现，但缺点是计算上比较费时间。

要正确理解误差反向传播法，有两种方法：

- 基于数学式
- 基于计算图（computational graph）

## 1.1 用计算图求解

**计算图** 是表达和评估数学表达式的一种方式，它被定义为有向图，其中节点对应于数学运算。

对于方程 $ z = x + y $ 的计算图如下：

![img](image/11.computational_graph.png)

问题1： 在超市买了 2 个 100 元一个的苹果，额外的消费税是 10%，请计算支付金额。

![img](image/11.apple_forward.png)

综上，用计算图解题的情况下，需要按如下流程进行。


1. 构建计算图。


2. 在计算图上，从左向右进行计算。

这里的第2 歩“从左向右进行计算”是正向传播（forward propagation）。正向传播是从计算图出发点到结束点的传播。而从右向左的传播被称为反向传播（backward propagation），它将在接下来的导数计算中发挥重要作用。

## 为何用计算图解题

- 无论全局是多么复杂的计算，都可以通过局部计算使各个节点致力于简单的计算，从而简化问题。


- 利用计算图可以将中间的计算结果全部保存起来。


- 可以通过反向传播高效计算导数。

这里，假设我们想知道苹果价格的上涨会在多大程度上影响最终的支付金额，即求“支付金额关于苹果的价格的导数”。设苹果的价格为 $x$，支付金额为 $L$，则相当于求 $\frac{\partial L}{\partial x}$。导数的数值可以通过计算图的反向传播求出来。


![img](image/11.apple_backward.png)

从这个结果中可知，“支付金额关于苹果的价格的导数”的值是2.2。这意味着，如果苹果的价格上涨 1 元，最终的支付金额会增加 2.2 元。

这里只求了关于苹果的价格的导数，不过“支付金额关于消费税的导数”、“支付金额关于苹果的个数的导数”等也都可以用同样的方式算出来。并且，计算中途求得的导数的结果（中间传递的导数）可以被共享，从而可以高效地计算多个导数。

综上，计算图的优点是，可以通过正向传播和反向传播高效地计算各个变量的导数值。

# 2. 链式法则

复合函数是由多个函数构成的函数。比如，$z = (x + y)^2$ 是由式（5.1）所示的两个式子构成的。

$$ z= t^2 $$
$$ t = x + y $$

如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

$z$ 关于 $x$ 的导数$ \frac{\partial z}{\partial x}$，可以用 $z$ 关于 $t$ 的导数 $\frac{\partial z}{\partial t}$ 和 $t$ 关于 $x$ 的导数 $\frac{\partial t}{\partial x}$ 的乘积表示。
$$ \frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t\cdot 1 = 2(x+y) $$

计算图的反向传播从右到左传播信号。反向传播的计算顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。

![img](image/11.chain1.png)

将求导结果带入上图中得到，
![img](image/11.chain2.png)

# 3. 反向传播

## 3.1 加法层的反向传播

以 $z = x + y$ 为对象，观察它的反向传播。$z = x + y$ 的导数可解析性地计算出来。

$$ \frac{\partial z}{\partial x} = 1$$

$$ \frac{\partial z}{\partial y} = 1$$

反向传播将从上游传过来的导数（本例中是$ \frac{\partial L}{\partial z}$）乘以1，然后传向下游。也就是说，因为加法节点的反向传播只乘以1，所以输入的值会原封不动地流向下一个节点。

![img](image\11.add_node.png)

接下来，我们实现加法节点的加法层，如下所示：

In [24]:
class AddLayer:
    def __init__(self):
        pass

    # 正向传播
    def forward(self, x, y):
        return x + y

    # 反向传播
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        # return dout, dout
        return dx, dy

加法层不需要特意进行初始化，所以 \_\_init\_\_() 中什么也不运行（pass语句表示“什么也不运行”）。加法层的 forward() 接收 x 和 y 两个参数，将它们相加后输出。backward() 将上游传来的导数（dout）原封不动地传递给下游。

## 3.2 乘法层的反向传播

这里我们考虑 $z = xy$。这个式的导数用下式表示：

$$ \frac{\partial z}{\partial x} = y$$

$$ \frac{\partial z}{\partial y} = x$$

乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。翻转值表示一种翻转关系，如下图所示，正向传播时信号是 x 的话，反向传播时则是 y；正向传播时信号是 y 的话，反向传播时则是 x。

![img](image/11.multi_node.png)

In [21]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    # 正向传播
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    # 反向传播
    def backward(self, dout):
        dx = dout * self.y # 翻转x和y
        dy = dout * self.x
        return dx, dy

\_\_init\_\_()中会初始化实例变量 x 和 y ，它们用于保存正向传播时的输入值。forward() 接收 x 和 y 两个参数，将它们相乘后输出。backward()将从上游传来的导数（dout）乘以正向传播的翻转值，然后传给下游。

现在我们使用 MulLayer 类实现前面的购买苹果的例子：

![img](image/11.apple_backward_result.png)

In [4]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

220.00000000000003


关于各个变量的导数可由backward()求出。这里调用backward()的顺序与调用forward()的顺序相反。

In [7]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
print(dapple_price, dtax)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num) # 2.2 110

1.1 200
2.2 110.00000000000001


## 3.3 激活函数层的反向传播

现在，我们将计算图的思路应用到神经网络中。这里，我们把构成神经网络的层实现为一个类。先来实现激活函数的 ReLU 层和 Sigmoid 层。

### 3.3.1 ReLU层

激活函数ReLU（Rectified Linear Unit）由下式表示，

$$ y = \begin{cases}
x&x > 0 \\
0&x <= 0
\end{cases} $$

可以求出 $y$ 关于 $x$ 的导数，

$$ \frac{\partial y}{\partial x} = \begin{cases}
1&x > 0 \\
0&x <= 0
\end{cases} $$

如果正向传播时的输入 $x$ 大于0，则反向传播会将上游的值原封不动地传给下游。反过来，如果正向传播时的 $x$ 小于等于0，则反向传播中传给下游的信号将停在此处。计算图如图所示，

![img](image/11.relu.png)

现在我们来实现ReLU层。在神经网络的层的实现中，一般假定 forward() 和 backward() 的参数是NumPy数组。

In [20]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        return dout

Relu 类有实例变量mask，这个变量是由 True/False 构成的 NumPy 数组，它会把正向传播时的输入 $x$ 的元素中小于等于0 的地方保存为 True，其它地方（大于0 的元素）保存为 False。

In [19]:
import numpy as np
x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )

Relu().forward(x)

array([[1., 0.],
       [0., 3.]])

### 3.3.2 Sigmoid层

sigmoid 函数公式为
$$ y = \frac{1}{1 + \exp (-x)} $$

用计算图表示的话如图所示，

![img](image/11.sigmoid.png)

下面我们进行计算图的反向传播。

- 步骤1
      “/”节点表示 y = 1/x，它的导数可以解析性地表示为下式。
$$\frac{\partial y}{\partial x} = - \frac{1}{x^2} = -y^2$$
- 步骤2
      “+”节点将上游的值原封不动地传给下游。
- 步骤3
      “exp”节点表示 y = exp(x)，它的导数为
$$ \frac{\partial y}{\partial x} = \exp(x) $$
- 步骤4
      “×”节点将正向传播时的值翻转后做乘法运算。因此，这里要乘以−1。

因此，Sigmoid层的反向传播计算图为：


![img](image/11.sigmoid_result.png)

Sigmoid 层的反向传播输出为 $\frac{\partial L}{\partial y}y^2\exp(-x)$，可以进一步整理如下：

$$\frac{\partial L}{\partial y}y^2\exp(-x) = \frac{\partial L}{\partial y}y\frac{1}{1 + \exp (-x)}\exp(-x) = \frac{\partial L}{\partial y}y(1-y)$$

![img](image/11.sigmoid_result_simple.png)

用 Python 代码实现sigmoid层：

In [28]:
class Sigmoid:
    def __init__(self):
        self.out = None    # 保存正向传播结果
        
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, dout):
        return dout * self.out * (1.0 - self.out)

## 3.4 Affine层

考虑一个简单的神经网络层计算：

In [31]:
X = np.random.rand(2) # 输入
W = np.random.rand(2,3) # 权重
B = np.random.rand(3) # 偏置

Y = np.dot(X, W) + B

这个计算过程可以用计算图表示出来。之前我们见到的计算图中各个节点间流动的是标量，而这个例子中各个节点间传播的是矩阵。

![img](image/11.affine_forward.png)

以矩阵为对象的反向传播，按矩阵的各个元素进行计算时，步骤和以标量为对象的计算图相同。

$$ \frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y}\cdot W^T $$
$$ \frac{\partial L}{\partial W} = X^T\cdot \frac{\partial L}{\partial Y} $$

神经网络计算图的反向传播：

![img](image/11.affine_backward.png)